# Catboost Pipeline with Librosa Features

In this notebook I show how to train catboost classifier using tabular data representing the features of audio signals. You can find the code for generating these features [here](https://www.kaggle.com/vadimtimakin/librosa-feature-generation).

I won't extract these features again here. They are already extracted and included in [this dataset](https://www.kaggle.com/vadimtimakin/librosa-features).

### Set up the enviroment

In [1]:
!pip install catboost

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier
from torch.nn import BCELoss

### Preparing data

In [3]:
df = pd.read_csv("../input/librosa-features/train_with_features.csv")
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
df.head()

,rating,playback_used,ebird_code,channels,date,pitch,duration,filename,speed,species,...,chroma_cens,melspectrogram,spectral_contrast,poly_features,tonnetz,tempogram,fourier_tempogram,rms,zero_crossing_rate,spectral_flatness
0,3.5,no,aldfly,1 (mono),2013-05-25,Not specified,25,XC134874.mp3,Not specified,Alder Flycatcher,...,0.281360,0.029290,19.055068,0.158539,0.002876,0.291609,0.543400,0.012144,0.097619,0.005510
1,4.0,no,aldfly,2 (stereo),2013-05-27,both,36,XC135454.mp3,both,Alder Flycatcher,...,0.280971,0.001065,15.744889,0.032611,-0.001276,0.165621,0.738268,0.002389,0.153774,0.026928
2,4.0,no,aldfly,2 (stereo),2013-05-27,both,39,XC135455.mp3,both,Alder Flycatcher,...,0.285239,0.036323,19.006677,0.267897,-0.003270,0.280726,0.501281,0.015590,0.201153,0.060953
3,3.5,no,aldfly,2 (stereo),2013-05-27,both,33,XC135456.mp3,both,Alder Flycatcher,...,0.276916,0.152413,16.775451,0.258931,0.002468,0.156127,0.686503,0.018586,0.159230,0.035714
4,4.0,no,aldfly,2 (stereo),2013-05-27,both,36,XC135457.mp3,level,Alder Flycatcher,...,0.273953,0.251364,19.860903,0.774564,0.004365,0.230685,0.673565,0.049090,0.168674,0.002025


In [5]:
columns =[
    "ebird_code",
    "chroma_stft",
    "spectral_centroid",
    "spectral_bandwidth",
    "spectral_rolloff",
    "mfcc",
    "chroma_cqt",
    "chroma_cens",
    "melspectrogram",
    "spectral_contrast",
    "poly_features",
    "tonnetz",
    "tempogram",
    "fourier_tempogram",
    "rms",
    "zero_crossing_rate",
    "spectral_flatness",
]

In [6]:
BIRD_CODE = {
    'aldfly': 0, 'ameavo': 1, 'amebit': 2, 'amecro': 3, 'amegfi': 4,
    'amekes': 5, 'amepip': 6, 'amered': 7, 'amerob': 8, 'amewig': 9,
    'amewoo': 10, 'amtspa': 11, 'annhum': 12, 'astfly': 13, 'baisan': 14,
    'baleag': 15, 'balori': 16, 'banswa': 17, 'barswa': 18, 'bawwar': 19,
    'belkin1': 20, 'belspa2': 21, 'bewwre': 22, 'bkbcuc': 23, 'bkbmag1': 24,
    'bkbwar': 25, 'bkcchi': 26, 'bkchum': 27, 'bkhgro': 28, 'bkpwar': 29,
    'bktspa': 30, 'blkpho': 31, 'blugrb1': 32, 'blujay': 33, 'bnhcow': 34,
    'boboli': 35, 'bongul': 36, 'brdowl': 37, 'brebla': 38, 'brespa': 39,
    'brncre': 40, 'brnthr': 41, 'brthum': 42, 'brwhaw': 43, 'btbwar': 44,
    'btnwar': 45, 'btywar': 46, 'buffle': 47, 'buggna': 48, 'buhvir': 49,
    'bulori': 50, 'bushti': 51, 'buwtea': 52, 'buwwar': 53, 'cacwre': 54,
    'calgul': 55, 'calqua': 56, 'camwar': 57, 'cangoo': 58, 'canwar': 59,
    'canwre': 60, 'carwre': 61, 'casfin': 62, 'caster1': 63, 'casvir': 64,
    'cedwax': 65, 'chispa': 66, 'chiswi': 67, 'chswar': 68, 'chukar': 69,
    'clanut': 70, 'cliswa': 71, 'comgol': 72, 'comgra': 73, 'comloo': 74,
    'commer': 75, 'comnig': 76, 'comrav': 77, 'comred': 78, 'comter': 79,
    'comyel': 80, 'coohaw': 81, 'coshum': 82, 'cowscj1': 83, 'daejun': 84,
    'doccor': 85, 'dowwoo': 86, 'dusfly': 87, 'eargre': 88, 'easblu': 89,
    'easkin': 90, 'easmea': 91, 'easpho': 92, 'eastow': 93, 'eawpew': 94,
    'eucdov': 95, 'eursta': 96, 'evegro': 97, 'fiespa': 98, 'fiscro': 99,
    'foxspa': 100, 'gadwal': 101, 'gcrfin': 102, 'gnttow': 103, 'gnwtea': 104,
    'gockin': 105, 'gocspa': 106, 'goleag': 107, 'grbher3': 108, 'grcfly': 109,
    'greegr': 110, 'greroa': 111, 'greyel': 112, 'grhowl': 113, 'grnher': 114,
    'grtgra': 115, 'grycat': 116, 'gryfly': 117, 'haiwoo': 118, 'hamfly': 119,
    'hergul': 120, 'herthr': 121, 'hoomer': 122, 'hoowar': 123, 'horgre': 124,
    'horlar': 125, 'houfin': 126, 'houspa': 127, 'houwre': 128, 'indbun': 129,
    'juntit1': 130, 'killde': 131, 'labwoo': 132, 'larspa': 133, 'lazbun': 134,
    'leabit': 135, 'leafly': 136, 'leasan': 137, 'lecthr': 138, 'lesgol': 139,
    'lesnig': 140, 'lesyel': 141, 'lewwoo': 142, 'linspa': 143, 'lobcur': 144,
    'lobdow': 145, 'logshr': 146, 'lotduc': 147, 'louwat': 148, 'macwar': 149,
    'magwar': 150, 'mallar3': 151, 'marwre': 152, 'merlin': 153, 'moublu': 154,
    'mouchi': 155, 'moudov': 156, 'norcar': 157, 'norfli': 158, 'norhar2': 159,
    'normoc': 160, 'norpar': 161, 'norpin': 162, 'norsho': 163, 'norwat': 164,
    'nrwswa': 165, 'nutwoo': 166, 'olsfly': 167, 'orcwar': 168, 'osprey': 169,
    'ovenbi1': 170, 'palwar': 171, 'pasfly': 172, 'pecsan': 173, 'perfal': 174,
    'phaino': 175, 'pibgre': 176, 'pilwoo': 177, 'pingro': 178, 'pinjay': 179,
    'pinsis': 180, 'pinwar': 181, 'plsvir': 182, 'prawar': 183, 'purfin': 184,
    'pygnut': 185, 'rebmer': 186, 'rebnut': 187, 'rebsap': 188, 'rebwoo': 189,
    'redcro': 190, 'redhea': 191, 'reevir1': 192, 'renpha': 193, 'reshaw': 194,
    'rethaw': 195, 'rewbla': 196, 'ribgul': 197, 'rinduc': 198, 'robgro': 199,
    'rocpig': 200, 'rocwre': 201, 'rthhum': 202, 'ruckin': 203, 'rudduc': 204,
    'rufgro': 205, 'rufhum': 206, 'rusbla': 207, 'sagspa1': 208, 'sagthr': 209,
    'savspa': 210, 'saypho': 211, 'scatan': 212, 'scoori': 213, 'semplo': 214,
    'semsan': 215, 'sheowl': 216, 'shshaw': 217, 'snobun': 218, 'snogoo': 219,
    'solsan': 220, 'sonspa': 221, 'sora': 222, 'sposan': 223, 'spotow': 224,
    'stejay': 225, 'swahaw': 226, 'swaspa': 227, 'swathr': 228, 'treswa': 229,
    'truswa': 230, 'tuftit': 231, 'tunswa': 232, 'veery': 233, 'vesspa': 234,
    'vigswa': 235, 'warvir': 236, 'wesblu': 237, 'wesgre': 238, 'weskin': 239,
    'wesmea': 240, 'wessan': 241, 'westan': 242, 'wewpew': 243, 'whbnut': 244,
    'whcspa': 245, 'whfibi': 246, 'whtspa': 247, 'whtswi': 248, 'wilfly': 249,
    'wilsni1': 250, 'wiltur': 251, 'winwre3': 252, 'wlswar': 253, 'wooduc': 254,
    'wooscj2': 255, 'woothr': 256, 'y00475': 257, 'yebfly': 258, 'yebsap': 259,
    'yehbla': 260, 'yelwar': 261, 'yerwar': 262, 'yetvir': 263
}

INV_BIRD_CODE = {k: v for k, v in BIRD_CODE.items()}

In [7]:
y = df.ebird_code.map(INV_BIRD_CODE)  # One-Hot encoding
df = df[columns]  # Leave features only 
df['ebird_code'] = y  # Target data
X = df.drop(['ebird_code'],axis=1)  # Train data
print(df)

       ebird_code  chroma_stft  spectral_centroid  spectral_bandwidth  \
0               0     0.407786        2530.639871         2691.628899   
1               0     0.408548        3623.059830         3342.633659   
2               0     0.531530        4656.412058         3753.303814   
3               0     0.440259        3560.451915         2831.272553   
4               0     0.378026        3246.539243         1799.140683   
...           ...          ...                ...                 ...   
21370         263     0.426162        6289.726690         2508.472647   
21371         263     0.468856        3698.690621         3271.603380   
21372         263     0.288500        2335.758772         2957.727400   
21373         263     0.624504        3711.789874         3007.188422   
21374         263     0.564780        4028.301572         3323.747565   

       spectral_rolloff       mfcc  chroma_cqt  chroma_cens  melspectrogram  \
0           3978.639069 -32.409058    0.5808

In [8]:
X

,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,mfcc,chroma_cqt,chroma_cens,melspectrogram,spectral_contrast,poly_features,tonnetz,tempogram,fourier_tempogram,rms,zero_crossing_rate,spectral_flatness
0,0.407786,2530.639871,2691.628899,3978.639069,-32.409058,0.580891,0.281360,0.029290,19.055068,0.158539,0.002876,0.291609,0.543400,0.012144,0.097619,0.005510
1,0.408548,3623.059830,3342.633659,7175.916570,-20.226545,0.635972,0.280971,0.001065,15.744889,0.032611,-0.001276,0.165621,0.738268,0.002389,0.153774,0.026928
2,0.531530,4656.412058,3753.303814,9611.458333,-28.381847,0.668917,0.285239,0.036323,19.006677,0.267897,-0.003270,0.280726,0.501281,0.015590,0.201153,0.060953
3,0.440259,3560.451915,2831.272553,6580.992594,-29.503727,0.570003,0.276916,0.152413,16.775451,0.258931,0.002468,0.156127,0.686503,0.018586,0.159230,0.035714
4,0.378026,3246.539243,1799.140683,5065.347813,-28.114975,0.516043,0.273953,0.251364,19.860903,0.774564,0.004365,0.230685,0.673565,0.049090,0.168674,0.002025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21370,0.426162,6289.726690,2508.472647,9103.898398,-31.546280,0.330717,0.208779,0.052880,20.496556,0.148400,0.003234,0.246456,0.237224,0.019537,0.343510,0.010983
21371,0.468856,3698.690621,3271.603380,7246.529956,-20.232605,0.548242,0.269998,0.002955,15.792201,0.038139,-0.026154,0.292593,0.579866,0.003717,0.159839,0.022823
21372,0.288500,2335.758772,2957.727400,4960.446363,-18.501550,0.539965,0.269653,0.002680,15.705705,0.044886,-0.011851,0.285615,0.514986,0.003629,0.071449,0.006469
21373,0.624504,3711.789874,3007.188422,6413.114364,-22.604296,0.763017,0.285161,0.006426,14.329976,0.036573,-0.004296,0.212859,0.575012,0.002826,0.185065,0.048411


In [9]:
y

0          0
1          0
2          0
3          0
4          0
        ... 
21370    263
21371    263
21372    263
21373    263
21374    263
Name: ebird_code, Length: 21375, dtype: int64

### Split the data into train and validation parts

In [10]:
train_data, eval_data, train_label, eval_label = train_test_split(
    X, y, test_size=0.2, random_state=42)
print(train_data.shape, eval_data.shape, train_label.shape, eval_label.shape)

(17100, 16) (4275, 16) (17100,) (4275,)


In [11]:
train_dataset = Pool(data=train_data,
                     label=train_label,
                     )

In [12]:
eval_dataset = Pool(data=eval_data,
                    label=eval_label,
                    )

### Create and set up the model

In [13]:
model = CatBoostClassifier(iterations=100,
                           learning_rate=0.003,
                           depth=10,
                           l2_leaf_reg = 0.01,
                           loss_function='MultiClass')

### Training

In [ ]:
model.fit(train_dataset)

### Predict

In [15]:
preds_class = model.predict(eval_dataset)
print(preds_class)

[[ 46]
 [177]
 [136]
 ...
 [ 12]
 [218]
 [129]]


In [16]:
preds = []
for i in range(len(preds_class)):
    labels = list([0] * len(BIRD_CODE))
    labels[preds_class[i][0]] = 1
    labels = np.array(labels)
    preds.append(labels)
preds = np.array(preds)

valid_labels = []
for i in range(len(eval_label)):
    labels = list([0] * len(BIRD_CODE))
    labels[eval_label.iloc[i]] = 1
    labels = np.array(labels)
    valid_labels.append(labels)
valid_labels = np.array(valid_labels)

### Counting the metric score

In [ ]:
from sklearn import metrics

fbeta_sklearn = metrics.fbeta_score(valid_labels, preds, 2, average='samples')
print(fbeta_sklearn)

### Saving the model

In [18]:
model.save_model("cbmodel.cbm",
           format="cbm",
           export_parameters=None,
           pool=None)